In [1]:
%load_ext autoreload
%autoreload 2
import gc
import re
import pandas as pd
import numpy as np
import os
import sys
import time
import datetime
import glob
sys.path.append('../py/')
from s027_kfold_ods import ods_kfold
HOME = os.path.expanduser("~")
sys.path.append(f'{HOME}/kaggle/data_analysis/library')
import utils
from utils import logger_func, get_categorical_features, get_numeric_features, reduce_mem_usage, elo_save_feature, impute_feature
try:
    if not logger:
        logger=logger_func()
except NameError:
    logger=logger_func()

from sklearn.metrics import mean_squared_error, roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor

#========================================================================
# Keras 
# Corporación Favorita Grocery Sales Forecasting
from sklearn.linear_model import Ridge
#========================================================================


#========================================================================
# Args
key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id', 'first_active_month', 'index', 'personal_term', 'no_out_flg', 'clf_pred']
start_time = "{0:%Y%m%d_%H%M%S}".format(datetime.datetime.now())
#========================================================================

2019-02-26 15:23:28,364 utils 340 [INFO]    [logger_func] start 


In [203]:
#========================================================================
# CV CHECKER 
# Data Load
def get_cv_score(base=[], model_path=''):
    
    pred_col = 'prediction'
    
    if len(base)==0:
        base = utils.read_df_pkl('../input/base_term*')[[key, target]].set_index(key)
    train = base[~base[target].isnull()]

    if len(model_path):
        tmp = utils.read_pkl_gzip(model_path)
        if 'pred_mean' in tmp.columns:
            pred_col = 'pred_mean'
        elif 'prediction' in tmp.columns:
            pred_col = 'prediction'
        tmp = tmp.reset_index()[[key, pred_col]].set_index(key)
        
        train[pred_col] = tmp[pred_col]

    y_train = train[target].values
    y_pred = train[pred_col].values
    score = np.sqrt(mean_squared_error(y_train, y_pred))
    print(f"CV: {score}")
    return score
#========================================================================

pred_col = 'prediction'
base = utils.read_pkl_gzip('../input/base_type_group.gz')[[key, target, 'type', 'no_out_flg', 'clf_pred']].set_index(key)
# base = utils.read_pkl_gzip('../input/base_type_group.gz')[[key, target]].set_index(key)

if 'type' in base.columns:
    min_thres = 0.01
    # Low Outlier Ratio
    type0_flg0_lower = ( (base['type']==0) & (base['no_out_flg']==0) ) & (base['clf_pred']<min_thres)
    type0_flg1_lower = ( (base['type']==0) & (base['no_out_flg']==1) ) & (base['clf_pred']<min_thres)
    type1_flg0_lower = ( (base['type']==1) & (base['no_out_flg']==0) ) & (base['clf_pred']<min_thres)
    type1_flg1_lower = ( (base['type']==1) & (base['no_out_flg']==1) ) & (base['clf_pred']<min_thres)
    type2_flg0_lower = ( (base['type']==2) & (base['no_out_flg']==0) ) & (base['clf_pred']<min_thres)
    type2_flg1_lower = ( (base['type']==2) & (base['no_out_flg']==1) ) & (base['clf_pred']<min_thres)
    
    # High Outlier Ratio
    type0_flg0_higher = ( (base['type']==0) & (base['no_out_flg']==0) ) & (base['clf_pred']>=min_thres)
    type0_flg1_higher = ( (base['type']==0) & (base['no_out_flg']==1) ) & (base['clf_pred']>=min_thres)
    type1_flg0_higher = ( (base['type']==1) & (base['no_out_flg']==0) ) & (base['clf_pred']>=min_thres)
    type1_flg1_higher = ( (base['type']==1) & (base['no_out_flg']==1) ) & (base['clf_pred']>=min_thres)
    type2_flg0_higher = ( (base['type']==2) & (base['no_out_flg']==0) ) & (base['clf_pred']>=min_thres)
    type2_flg1_higher = ( (base['type']==2) & (base['no_out_flg']==1) ) & (base['clf_pred']>=min_thres)
    type10 = (base['type']==10)
    base.drop('clf_pred', axis=1, inplace=True)
    
cv3595 = utils.read_pkl_gzip('../ensemble/good_submit_ensemble/tmp/0227_alijs_stack_submit_CV3-5956_LB3-500.gz')[[key, pred_col]].set_index(key).rename(columns={pred_col:'prediction_1'})
cv3601 = utils.read_pkl_gzip('../ensemble/good_submit_ensemble/tmp/0226_184_stack_submit_log_outlier_processing_blend_base1_clf_out0_rm_out1_no_out_blend0_minthres0.01_OUT1.8463-29.638_CV3.60187_LB.gz')[[key, pred_col]].set_index(key).rename(columns={pred_col:'prediction_2'})
cv3606 = utils.read_pkl_gzip('../ensemble/good_submit_ensemble/tmp/0227_071_stack_submit_log_outlier_processing_blend_base1_clf_out0_rm_out1_no_out_blend0_minthres0.01_OUT1.8500-29.669_CV3.60645_LB.gz')[[key, pred_col]].set_index(key).rename(columns={pred_col:'prediction_3'})
lb3663 = utils.read_pkl_gzip('../ensemble/good_submit_ensemble/tmp/0224_080_stack_submit_log_outlier_processing_blend_base1_clf_out0_rm_out1_no_out_blend0_minthres0.01_OUT1.8537-29.651_CV3.60681_LB3.663.gz')[[key, pred_col]].set_index(key).rename(columns={pred_col:'prediction_4'})
lb3667 = utils.read_pkl_gzip('../ensemble/good_submit_ensemble/tmp/0213_074_stack_submit_lgb8_rm_out-lgb1_no_out-0_clf-thres0.0082_CV3.6088898700520273_LB3.667.gz')[[key, pred_col]].set_index(key).rename(columns={pred_col:'prediction_5'})
lb3660 = pd.read_csv('../ensemble/good_submit_ensemble/tmp/alijs_stack_submit_train_02210021_CV3.6066_LB3.660.csv')[[key, pred_col]].set_index(key).rename(columns={pred_col:'prediction_6'})
lb3663_2 = pd.read_csv('../ensemble/good_submit_ensemble/tmp/aljis_stack_submit_train_02202319_CV3.6003_LB3.663.csv')[[key, pred_col]].set_index(key).rename(columns={pred_col:'prediction_7'})

base = base.join(cv3597).join(cv3601).join(cv3606).join(lb3663).join(lb3667).join(lb3660).join(lb3663_2)
pred_col_list = [col for col in base.columns if col.count('pred')]

print(pred_col_list)
# display(base.head())

base[pred_col] = base[pred_col_list[0]]*0.70 + base[pred_col_list[1]]*0.30
base.loc[type0_flg0_higher, pred_col] = base.loc[type0_flg0_higher, pred_col_list[0]]*0.4 + base.loc[type0_flg0_higher, pred_col_list[1]]*0.3 + base.loc[type0_flg0_higher, pred_col_list[2]]*0.2 + base.loc[type0_flg0_higher, pred_col_list[6]]*0.1
base.loc[type0_flg0_lower, pred_col] = base.loc[type0_flg0_lower, pred_col_list[0]]*0.35 + base.loc[type0_flg0_lower, pred_col_list[1]]*0.2 + base.loc[type0_flg0_lower, pred_col_list[2]]*0.0 + base.loc[type0_flg0_lower, pred_col_list[3]]*0.2 + base.loc[type0_flg0_lower, pred_col_list[4]]*0.1 + base.loc[type0_flg0_lower, pred_col_list[6]]*0.15
base.loc[type1_flg1_lower, pred_col] = base.loc[type1_flg1_lower, pred_col_list[0]]*0.5 + base.loc[type1_flg1_lower, pred_col_list[1]]*0.2 + base.loc[type1_flg1_lower, pred_col_list[6]]*0.3
base.loc[type2_flg0_higher, pred_col] = base.loc[type2_flg0_higher, pred_col_list[0]]*0.60 + base.loc[type2_flg0_higher, pred_col_list[1]]*0.05 + base.loc[type2_flg0_higher, pred_col_list[3]]*0.05 + base.loc[type2_flg0_higher, pred_col_list[4]]*0.0 + base.loc[type2_flg0_higher, pred_col_list[6]]*0.3
base.loc[type2_flg0_lower, pred_col] = base.loc[type2_flg0_lower, pred_col_list[0]]*0.6 + base.loc[type2_flg0_lower, pred_col_list[1]]*0.1 + base.loc[type2_flg0_lower, pred_col_list[6]]*0.3
base.loc[type10, pred_col] = base.loc[type10, pred_col_list[0]]*0.5 + base.loc[type10, pred_col_list[1]]*0.2 + base.loc[type10, pred_col_list[3]]*0.05 + base.loc[type10, pred_col_list[4]]*0.0 + base.loc[type10, pred_col_list[5]]*0.1 + base.loc[type10, pred_col_list[6]]*0.15

cv_score = get_cv_score(base=base)

display(base.head(2))
display(base.tail(2))

['prediction_1', 'prediction_2', 'prediction_3', 'prediction_4', 'prediction_5', 'prediction_6', 'prediction_7']
CV: 3.5965442018101226


,target,type,no_out_flg,prediction_1,prediction_2,prediction_3,prediction_4,prediction_5,prediction_6,prediction_7,prediction
card_id,,,,,,,,,,,
C_ID_92a2005557,-0.8202826000,2,1.0000000000,-0.3001041509,-0.3217995967,-0.3217995967,-0.3070135371,-0.3350633585,-0.3197117613,-0.3069228403,-0.3066127846
C_ID_3d0044924f,0.3929132500,0,0.0000000000,-0.0552443518,0.1475861396,0.1475861396,0.1859042236,0.3744892201,0.2527631566,-0.0210684936,0.0816511975


,target,type,no_out_flg,prediction_1,prediction_2,prediction_3,prediction_4,prediction_5,prediction_6,prediction_7,prediction
card_id,,,,,,,,,,,
C_ID_6c46fc5a9d,nan,0,0.0000000000,-3.4806135759,-3.6241232382,-3.5581009965,-3.5927115406,-3.9307435943,-3.6489566758,-3.5242436545,-3.5435269666
C_ID_87e7979a5f,nan,2,0.0000000000,0.3228795935,0.3274292719,0.3274292719,0.3159261731,0.4555413303,0.3215417152,0.3279818155,0.3248652280


In [204]:
submit = pd.read_csv('../input/sample_submission.csv').set_index(key)
submit[target] = base[pred_col]
start_time = "{0:%Y%m%d_%H%M%S}".format(datetime.datetime.now())
submit_path = f'{start_time[4:12]}_submit_blend_5Best_Model_last_submit_2.csv'
submit.to_csv(f'../submit/{submit_path}')
utils.to_pkl_gzip(obj=base.reset_index()[[key, target, 'prediction']], path=f"../stack/{submit_path.replace('submit', 'stack_submit_log')}")
display(submit.head())

,target
card_id,
C_ID_0ab67a22ab,-1.4099019824
C_ID_130fd0cbdd,-0.1569349619
C_ID_b709037bc5,-0.8353908672
C_ID_d27d835a9f,-0.1056215640
C_ID_2b5e3df5c2,-1.3312244497


In [156]:
raw = 'raw_pred'
base[raw] = base[pred_col].map(lambda x: 2**x)
base[raw] = base[raw].map(lambda x: np.round(x, 1))
# base[raw] = base[pred_col].map(lambda x: np.round(x, 3))
id_list = list(base[base[raw]==1].index)
print(len(id_list))
base.loc[id_list, pred_col] = 0
id_list = list(base[base[raw]==2].index)
print(len(id_list))
base.loc[id_list, pred_col] = 1
get_cv_score(base=base)

20334
1973
CV: 3.596266407811942


3.596266407811942

In [114]:
# a: cv3.597
# B: mine(CV3.601)
# type0_flg0_higher: A*0.5  + B*0.5
# type2_flg0_higher: A*0.95 + B*0.05
# type2_flg0_lower : A*1.0  + B*0.0
# other: A*0.7 + B*0.3

In [141]:
# for i in np.arange(0.1, 100, 0.1):
#     print(np.log2(i))
print(np.log2(2))

nan


In [94]:
train = pd.read_csv('../input/train.csv')
train['new_target'] = 2**train[target]

pd.set_option('display.float_format', '{:.10f}'.format)
pd.set_option('max_rows', 100)
train.groupby('new_target')[key].count().sort_values(ascending=False).head(50)

new_target
0.0000000001    2207
1.0000000000    1630
2.0000000000     117
0.5000000000      81
0.6666666670      35
1.4999999993      28
0.4000000014      19
1.2183406082      17
2.9999999985      14
1.2675585249      13
0.7499999996      12
1.1179941041      11
0.8000000027      11
0.3333333335      11
0.8333333309      10
4.0000000000      10
1.2499999958      10
1.3333333340       8
1.1091703067       8
2.4999999915       8
0.2500000000       8
1.6666666619       8
1.7499999975       7
0.6000000017       7
0.7000000014       7
0.2000000007       7
0.1000000003       7
1.4285714258       7
1.1337792650       6
0.8571428579       6
2.2499999978       6
1.0526315779       6
4.9999999831       6
1.0984252006       6
0.4545454557       6
0.4761904755       5
3.3333333237       5
2.4000000069       5
0.9869791684       5
1.0999999971       5
0.9523809510       5
1.6000000054       5
0.2222222224       5
1.3584229425       5
0.3000000009       5
0.9998680889       5
0.6060606079       4
0.